In [ ]:
import os
import sys
import numpy as np
from nanowire.optics.simulate import Simulator
from nanowire.optics.postprocess import Simulation
from nanowire.optics.utils.utils import setup_sim
from nanowire.optics.utils.config import Config
from nanowire.optics.utils.geometry import *
from itertools import accumulate, repeat, chain
import scipy.constants as consts
import scipy.integrate as intg
import scipy.interpolate as spi
import IPython.display as disp
import matplotlib.pyplot as plt
plt.style.use(['presentation'])
%load_ext autoreload 
%autoreload 2
%load_ext ipycache

In [ ]:
local_vars = locals()
mem_useage = [(key, sys.getsizeof(val)/1024) for key, val in local_vars.items()]
total = 0
for name, val in sorted(mem_useage, key=lambda tup: tup[1], reverse=True):
    print("{}: {} Kb".format(name, val))
    total += val
print('Total Useage: {} Mb'.format(total/1024))

# Utility Functions

In [ ]:
def integrate1d(arr, xvals, meth=intg.trapz):
    x_integral = meth(arr, x=xvals, axis=0)
    return x_integral

def integrate2d(arr, xvals, yvals, meth=intg.trapz):
    x_integral = meth(arr, x=xvals, axis=0)
    y_integral = meth(x_integral, x=yvals, axis=0)
    return y_integral

def integrate3d(arr, xvals, yvals, zvals, meth=intg.trapz):
    ##print("Layer: {}".format(layer_obj.name))
    ##print("Layer Start Ind: {}".format(layer_obj.istart))
    ##print("Layer End Ind: {}".format(layer_obj.iend))
    ##print(z_vals)
    z_integral = meth(arr, x=zvals, axis=0)
    x_integral = meth(z_integral, x=xvals, axis=0)
    y_integral = meth(x_integral, x=yvals, axis=0)
    return y_integral

def compute_fluxes(sim):
    fluxes = sim.data['fluxes']
    total_incident_power = .5*sim.period**2/Zo*np.absolute(fluxes['Air'][0])
    total_reflected_power = .5*sim.period**2/Zo*np.absolute(fluxes['Air'][1])
    total_transmitted_power = .5*sim.period**2/Zo*np.absolute(sum(fluxes['Substrate_bottom']))
    total_absorbed_power = total_incident_power - total_reflected_power - total_transmitted_power
    #print('Total Incident Power = {}'.format(total_incident_power))
    #print('Total Reflected Power = {}'.format(total_reflected_power))
    #print('Total Transmitted Power = {}'.format(total_transmitted_power))
    #print('Total Absorbed Power = {}'.format(total_absorbed_power))
    summed_absorbed_power = 0
    abs_dict_fluxmethod = {}
    for layer, (forw_top, back_top) in fluxes.items():
        if '_bottom' in layer:
            continue
        bottom = layer+'_bottom'
        forw_bot, back_bot = fluxes[bottom] 
        #print('-'*25)
        #print('Layer: {}'.format(layer))
        #print('Forward Top: {}'.format(forw_top))
        #print('Backward Top: {}'.format(back_top))
        #print('Forward Bottom: {}'.format(forw_bot))
        #print('Backward Bottom: {}'.format(back_bot))
        P_in = forw_top + -1*back_bot
        P_out = forw_bot + -1*back_top
        #print('Power Entering Layer: {}'.format(P_in))
        #print('Power Leaving Layer: {}'.format(P_out))
        P_lost = P_in - P_out
        P_abs = .5*P_lost*(sim.period**2)/Zo
        abs_dict_fluxmethod[layer] = P_abs 
        #print('Absorbed in Layer: {}'.format(P_abs))
        summed_absorbed_power += P_abs
    #print('-'*25)
    #print('Summed Absorption= {}'.format(summed_absorbed_power))
    return abs_dict_fluxmethod, summed_absorbed_power

def get_intmethod_abs(sim_proc):
    abs_dict_intmethod = {}
    int_method_total = 0
    freq = sim_proc.conf[("Simulation","params","frequency","value")]
    Esq = sim_proc.normEsquared()
    for layer_name, layer_obj in sim_proc.layers.items():
        #print("Layer: {}".format(layer_name))
        base_unit = sim_proc.conf[('Simulation', 'base_unit')]
        n_mat, k_mat = layer_obj.get_nk_matrix(freq)
        # n and k could be functions of space, so we need to multiply the
        # fields by n and k before integrating
        arr = Esq[layer_obj.slice]
        z_vals = sim_proc.Z[layer_obj.istart:layer_obj.iend]
        res = integrate3d(arr*n_mat*k_mat,sim_proc.X, sim_proc.Y, z_vals)
        if np.isnan(res):
            #print("Result is nan!")
            res = 0
        p_abs_imag = 2*np.pi*freq*consts.epsilon_0*res*base_unit
        abs_dict_intmethod[layer_name] = p_abs_imag
        #disp.display_latex("$P_{abs} = \\frac{\omega}{2} Im(\epsilon) \int |E|^2 dV"+" = {}$".format(p_abs_imag), raw=True)
        int_method_total += p_abs_imag
    #print("Integral Method Total Absorption: {}".format(int_method_total))
    return abs_dict_intmethod, int_method_total

def get_errors(abs_dict_intmethod, abs_dict_fluxmethod, int_method_total, flux_method_total):
    errors = {}
    for key in abs_dict_fluxmethod.keys():
        fm = abs_dict_fluxmethod[key]
        im = abs_dict_intmethod[key]
        diff = np.real(fm - im)
        try:
            pdiff = 100*abs(diff)/fm
        except ZeroDivisionError:
            pdiff = None
            pass
        #print('-'*25)
        #print("Layer: {}".format(key))
        #print("Flux Method: {}".format(fm))
        #print("Integral Method: {}".format(im))
        #print("Diff: {}".format(diff))
        #print("Percent Diff: {}".format(pdiff))
        errors[key] = (diff, pdiff)
    pdiff_total = 100*abs(flux_method_total - int_method_total)/flux_method_total
    #print('-'*25)
    #print("Total Percent Difference: {}".format(pdiff_total))
    errors['total'] = pdiff_total
    return errors

def get_layer_absorption(sim, layer, plane_samps, zsamps, interp='nknormEsq', meth=intg.trapz):
    layer_results = []
    freq = sim.conf[('Simulation', 'params', 'frequency')]
    base_unit = sim.conf[('Simulation', 'base_unit')]
    period = sim.conf[('Simulation', 'params', 'array_period')]
    layer_obj = sim.layers[layer]
    if interp == 'nknormEsq':
        x = np.linspace(0, period, plane_samps)
        z = np.linspace(layer_obj.start, layer_obj.end, zsamps)
        pts = cartesian_product((z, x, x))
        nkEsq = sim.nknormEsq(pts)
        nkEsq = nkEsq.reshape((zsamps, plane_samps, plane_samps))
        result = integrate3d(nkEsq, x, x, z, meth=meth)
    elif interp == 'normEsq':
        x = np.linspace(0, period, plane_samps)
        n_mat, k_mat = layer_obj.get_nk_matrix(freq, xcoords=x, ycoords=x)
        z = np.linspace(layer_obj.start, layer_obj.end, zsamps)
        pts = cartesian_product((z, x, x))
        Esq = sim.normEsq(pts)
        Esq = Esq.reshape((zsamps, plane_samps, plane_samps))
        result = integrate3d(n_mat*k_mat*Esq, x, x, z, meth=meth)
    elif interp == 'nknormEsqno':
        n_mat, k_mat = layer_obj.get_nk_matrix(freq, xcoords=sim.X, ycoords=sim.Y)
        z = sim.Z[layer_obj.slice]
        Esq = sim.data['nknormEsq'][layer_obj.slice]
        result = integrate3d(Esq, sim.X, sim.Y, z, meth=meth)
    elif interp == 'normEsqno':
        z = sim.Z[layer_obj.slice]
        n_mat, k_mat = layer_obj.get_nk_matrix(freq, xcoords=sim.X, ycoords=sim.Y)
        Esq = sim.data['normEsq'][layer_obj.slice]
        result = integrate3d(n_mat*k_mat*Esq, sim.X, sim.Y, z, meth=meth)
    else:
        raise ValueError('Invalid interp method')
    result = 2*np.pi*freq*consts.epsilon_0*result*base_unit
    print("Done with get_layer_absorption!")
    return result

def cartesian_product(arrays, out=None):
    la = len(arrays)
    L = *map(len, arrays), la
    dtype = np.result_type(*arrays)
    arr = np.empty(L, dtype=dtype)
    arrs = *accumulate(chain((arr,), repeat(0, la-1)), np.ndarray.__getitem__),
    idx = slice(None), *itertools.repeat(None, la-1)
    for i in range(la-1, 0, -1):
        arrs[i][..., i] = arrays[i][idx[:la-i]]
        arrs[i-1][1:] = arrs[i]
    arr[..., 0] = arrays[0][idx]
    return arr.reshape(-1, la)

# Analysis

## Get the data

In [ ]:
conf = Config('interpolator.yml')
sim = Simulator(conf)
sim.setup()
sim.get_layers()
Zo = consts.physical_constants['characteristic impedance of vacuum'][0]
%time sim.load_state()

In [ ]:
%time sim.get_fluxes()
sim.data['fluxes'] = {record[0].decode():(record[1], record[2]) for record in sim.data['fluxes']}
abs_dict_fluxmethod, flux_method_total = compute_fluxes(sim)
print("######")
%time sim.save_state()

In [ ]:
%%time
sim.get_field()

In [ ]:
%%time
normEsq = np.abs(sim.data['Ex'])**2 + np.abs(sim.data['Ey'])**2 + np.abs(sim.data['Ez'])**2
sim.data['normEsq'] = normEsq
nknormEsq = np.zeros_like(normEsq)
freq = sim.conf['Simulation']['params']['frequency']
for name, layer in sim.layers.items():
    n, k = layer.get_nk_matrix(freq)
    nknormEsq[layer.slice] = normEsq[layer.slice]*n*k
sim.data['nknormEsq'] = nknormEsq
del normEsq
del nknormEsq

In [ ]:
sim.add_interpolator('nknormEsq')
sim.add_interpolator('normEsq')

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 18))
ax1.matshow(sim.data['nknormEsq'][:, :, 150])
ax2.matshow(sim.data['normEsq'][:, :, 150])
plt.show()

## Bivariate Spline in Planes, nknormEsq

In [ ]:
# Integrate nanowire core
zsamps = 300
plane_samps = 500
zsamps = sim.conf['Simulation']['z_samples']
period = sim.conf['Simulation']['params']['array_period']
freq = sim.conf['Simulation']['params']['frequency']
base_unit = sim.conf[('Simulation', 'base_unit')]
x = np.linspace(0, period, plane_samps)
plane_integrals = np.zeros(zsamps)
for i in range(zsamps):
    plane_data = sim.data['nknormEsq'][i,:,:]
    rbs = spi.RectBivariateSpline(sim.X, sim.Y, plane_data)
    presult = rbs.integral(sim.X[0], sim.X[-1], sim.Y[0], sim.Y[-1])
    #presult = integrate2d(vals, r_vals, theta_vals)
    #presult = integrate2d(vals, theta_vals, r_vals)
    plane_integrals[i] = presult

In [ ]:
z_int = integrate1d(plane_integrals, sim.Z)
result = 2*np.pi*freq*consts.epsilon_0*z_int*base_unit
flux_result = abs_dict_fluxmethod['NW_AlShell'] 
print("Integral Result: {}".format(result))
print("Flux Result: {}".format(flux_result))
diff = np.abs(result - abs_dict_fluxmethod['NW_AlShell'])/np.abs(abs_dict_fluxmethod['NW_AlShell'])
print("Diff: {}".format(diff))

## Bivariate Spline in Planes, normEsq

In [ ]:
# Integrate nanowire core
zsamps = 300
plane_samps = 500
zsamps = sim.conf['Simulation']['z_samples']
period = sim.conf['Simulation']['params']['array_period']
freq = sim.conf['Simulation']['params']['frequency']
base_unit = sim.conf[('Simulation', 'base_unit')]
plane_integrals = np.zeros(zsamps)
layer_obj = sim.layers['NW_AlShell']
nmat, kmat = layer_obj.get_nk_matrix(freq, sim.X, sim.Y)
for i in range(zsamps):
    plane_data = sim.data['normEsq'][i,:,:]
    rbs = spi.RectBivariateSpline(sim.X, sim.Y, nmat*kmat*plane_data)
    presult = rbs.integral(sim.X[0], sim.X[-1], sim.Y[0], sim.Y[-1])
    #presult = integrate2d(vals, r_vals, theta_vals)
    #presult = integrate2d(vals, theta_vals, r_vals)
    plane_integrals[i] = presult

In [ ]:
z_int = integrate1d(plane_integrals, sim.Z)
result = 2*np.pi*freq*consts.epsilon_0*z_int*base_unit
flux_result = abs_dict_fluxmethod['NW_AlShell'] 
print("Integral Result: {}".format(result))
print("Flux Result: {}".format(flux_result))
diff = np.abs(result - abs_dict_fluxmethod['NW_AlShell'])/np.abs(abs_dict_fluxmethod['NW_AlShell'])
print("Diff: {}".format(diff))

## 3D Linear Interpolation, nknormEsq

In [ ]:
%%time
result = get_layer_absorption(sim, 'NW_AlShell', 400, 50, interp='nknormEsq')
flux_result = abs_dict_fluxmethod['NW_AlShell'] 
print("Integral Result: {}".format(result))
print("Flux Result: {}".format(flux_result))
diff = np.abs(result - abs_dict_fluxmethod['NW_AlShell'])/np.abs(abs_dict_fluxmethod['NW_AlShell'])
print("Diff: {}".format(diff))

In [ ]:
%%time
result = get_layer_absorption(sim, 'NW_AlShell', 100, 50, interp='nknormEsq', meth=intg.simps)
flux_result = abs_dict_fluxmethod['NW_AlShell'] 
print("Integral Result: {}".format(result))
print("Flux Result: {}".format(flux_result))
diff = np.abs(result - abs_dict_fluxmethod['NW_AlShell'])/np.abs(abs_dict_fluxmethod['NW_AlShell'])
print("Diff: {}".format(diff))

So it seems like fewer sampling points actually enchances the agreement between the two methods for calculating
absorption. 

## nknormEsq, no interpolation 

In [ ]:
%%time
result = get_layer_absorption(sim, 'NW_AlShell', 100, 50, interp='nknormEsqno')
flux_result = abs_dict_fluxmethod['NW_AlShell'] 
print("Integral Result: {}".format(result))
print("Flux Result: {}".format(flux_result))
diff = np.abs(result - abs_dict_fluxmethod['NW_AlShell'])/np.abs(abs_dict_fluxmethod['NW_AlShell'])
print("Diff: {}".format(diff))

In [ ]:
%%time
result = get_layer_absorption(sim, 'NW_AlShell', 100, 50, interp='nknormEsqno', meth=intg.simps)
flux_result = abs_dict_fluxmethod['NW_AlShell'] 
print("Integral Result: {}".format(result))
print("Flux Result: {}".format(flux_result))
diff = np.abs(result - abs_dict_fluxmethod['NW_AlShell'])/np.abs(abs_dict_fluxmethod['NW_AlShell'])
print("Diff: {}".format(diff))

## 3D Linear Interpolation, normEsq

In [ ]:
%%time
result = get_layer_absorption(sim, 'NW_AlShell', 300, 200, interp='normEsq')
flux_result = abs_dict_fluxmethod['NW_AlShell'] 
print("Integral Result: {}".format(result))
print("Flux Result: {}".format(flux_result))
diff = np.abs(result - abs_dict_fluxmethod['NW_AlShell'])/np.abs(abs_dict_fluxmethod['NW_AlShell'])
print("Diff: {}".format(diff))

In [ ]:
%%time
result = get_layer_absorption(sim, 'NW_AlShell', 300, 200, interp='normEsq', meth=intg.simps)
flux_result = abs_dict_fluxmethod['NW_AlShell'] 
print("Integral Result: {}".format(result))
print("Flux Result: {}".format(flux_result))
diff = np.abs(result - abs_dict_fluxmethod['NW_AlShell'])/np.abs(abs_dict_fluxmethod['NW_AlShell'])
print("Diff: {}".format(diff))

## normEsq, no interpolation 

In [ ]:
%%time
result = get_layer_absorption(sim, 'NW_AlShell', 100, 50, interp='normEsqno')
flux_result = abs_dict_fluxmethod['NW_AlShell'] 
print("Integral Result: {}".format(result))
print("Flux Result: {}".format(flux_result))
diff = np.abs(result - abs_dict_fluxmethod['NW_AlShell'])/np.abs(abs_dict_fluxmethod['NW_AlShell'])
print("Diff: {}".format(diff))

In [ ]:
%%time
result = get_layer_absorption(sim, 'NW_AlShell', 100, 50, interp='normEsqno', meth=intg.simps)
flux_result = abs_dict_fluxmethod['NW_AlShell'] 
print("Integral Result: {}".format(result))
print("Flux Result: {}".format(flux_result))
diff = np.abs(result - abs_dict_fluxmethod['NW_AlShell'])/np.abs(abs_dict_fluxmethod['NW_AlShell'])
print("Diff: {}".format(diff))

## Integrate By Material

In [ ]:
# Integrate nanowire core
zsamps = 200
period = sim.conf['Simulation']['params']['array_period']
freq = sim.conf['Simulation']['params']['frequency']
base_unit = sim.conf[('Simulation', 'base_unit')]
layer_obj = sim.layers["NW_AlShell"]
z_vals = np.linspace(layer_obj.start, layer_obj.end, zsamps)
plane_integrals = np.zeros(zsamps)
core_rad = sim.conf[('Layers','NW_AlShell','geometry','core','radius')]
shell_rad = sim.conf[('Layers','NW_AlShell','geometry','shell','radius')]
r_vals = np.linspace(.0000001, core_rad, 60, endpoint=False)
theta_vals = np.linspace(0, 2*np.pi, 180, endpoint=False)
pts = cartesian_product((np.array([z_vals]), r_vals, theta_vals))
xy_pts = np.zeros_like(pts)
xy_pts[:, 1] = pts[:, 1] * np.cos(pts[:,2]) + period/2
xy_pts[:, 2] = pts[:, 1] * np.sin(pts[:,2]) + period/2
xy_pts[:, 0] = pts[:, 0]
vals = sim.nknormEsq(xy_pts)
vals = vals.reshape((len(r_vals), len(theta_vals)))
#vals = vals.reshape((len(theta_vals), len(r_vals)))
presult = integrate2d(vals, r_vals, theta_vals)
#presult = integrate2d(vals, theta_vals, r_vals)
plane_integrals[i] = presult
#for i, z in enumerate(z_vals):
#    #n_mat, k_mat = layer_obj.get_nk_matrix(freq, xcoords=x, ycoords=y)
#    r_vals = np.linspace(.0000001, core_rad, 60, endpoint=False)
#    theta_vals = np.linspace(0, 2*np.pi, 180, endpoint=False)
#    pts = cartesian_product((np.array([z]), r_vals, theta_vals))
#    xy_pts = np.zeros_like(pts)
#    xy_pts[:, 1] = pts[:, 1] * np.cos(pts[:,2]) + period/2
#    xy_pts[:, 2] = pts[:, 1] * np.sin(pts[:,2]) + period/2
#    xy_pts[:, 0] = pts[:, 0]
#    vals = sim.nknormEsq(xy_pts)
#    vals = vals.reshape((len(r_vals), len(theta_vals)))
#    #vals = vals.reshape((len(theta_vals), len(r_vals)))
#    presult = integrate2d(vals, r_vals, theta_vals)
#    #presult = integrate2d(vals, theta_vals, r_vals)
#    plane_integrals[i] = presult

In [ ]:
z_int = integrate1d(plane_integrals, z_vals)
result = 2*np.pi*freq*consts.epsilon_0*z_int*base_unit
flux_result = abs_dict_fluxmethod['NW_AlShell'] 
print("Integral Result: {}".format(result))
print("Flux Result: {}".format(flux_result))
diff = np.abs(result - abs_dict_fluxmethod['NW_AlShell'])/np.abs(abs_dict_fluxmethod['NW_AlShell'])
print("Diff: {}".format(diff))

In [ ]:
#nk_dict = layer_obj.get_nk_dict(freq)
#for name, (shape, mat) in layer_obj.shapes.items():
#    mask = get_mask(shape, x, x)
#    print(name)
#    n, k = nk_dict[mat] 
#    print(n, k)
#    mask = mask*n*k
#    plt.matshow(mask)
#    plt.show()
n_mat, k_mat = layer_obj.get_nk_matrix(freq, xcoords=x, ycoords=y)
core_rad = sim.conf[('Layers','NW_AlShell','geometry','core','radius')]
shell_rad = sim.conf[('Layers','NW_AlShell','geometry','shell','radius')]
r_vals = np.linspace(.0000001, core_rad, 60)
theta_vals = np.linspace(0, 2*np.pi, 180, endpoint=False)
pts = cartesian_product((r_vals, theta_vals, z_vals))
print(pts.shape)
xy_pts = np.zeros_like(pts)
xy_pts[:, 1] = pts[:, 0] * np.cos(pts[:,1]) + .25/2
xy_pts[:, 2] = pts[:, 0] * np.sin(pts[:,1]) + .25/2
xy_pts[:, 0] = pts[:, 2]
vals = sim.normEsq(xy_pts)

Z changes, then y, then x